# Creating Resource Tables Using Patient As an Example 
***

In [0]:
DECLARE OR REPLACE VARIABLE schema_use STRING DEFAULT 'primary_clinical_data'

In [0]:
USE IDENTIFIER("nebraska_health_demo." || schema_use);

Determine Column Names to Pivot and Dynamically Set DDL

The column names are required for our `PIVOT`

In [0]:
DECLARE OR REPLACE VARIABLE patient_keys STRING;

SET VAR patient_keys = 
(
SELECT array_join(collect_list(DISTINCT column_name), "', '") as column_name_list
FROM fhir_bronze_resource_schemas
WHERE resourceType = 'Patient'
)

In [0]:
DECLARE OR REPLACE VARIABLE patient_dynamic_ddl STRING;

SET VAR patient_dynamic_ddl = (
  SELECT
    "\tpatient_uuid STRING NOT NULL PRIMARY KEY COMMENT 'Unique identifier for the FHIR Resource in a bundle.'\n\t,bundle_uuid STRING COMMENT 'Unique identifier for the FHIR bundle.'\n\t,patient_url STRING COMMENT 'The full URL of the resource in the entry array.  This serves as the primary key for the resource and later is used to join related resources from within a bundle.'\n\t," || 
    array_join(collect_list(DISTINCT column_name), " VARIANT\n\t,") || " VARIANT\n\t" ||
    ",CONSTRAINT fk_patient_bundle_meta_bundle_uuid FOREIGN KEY (bundle_uuid) REFERENCES nebraska_health_demo." || schema_use || ".bundle_metadata(bundle_uuid)
    \t,CONSTRAINT fk_patient_fhir_resources_resource_uuid FOREIGN KEY (patient_uuid) REFERENCES nebraska_health_demo." || schema_use || ".fhir_bronze_resources(resource_uuid)"
  FROM 
    fhir_bronze_resource_schemas
  WHERE 
    resourceType = 'Patient'
);

select patient_dynamic_ddl;

In [0]:
DROP TABLE IF EXISTS patient;

In [0]:
DECLARE OR REPLACE VARIABLE patient_resource_stmnt STRING;

SET VARIABLE patient_resource_stmnt = "
CREATE OR REFRESH STREAMING TABLE patient (\n" ||
  patient_dynamic_ddl
|| "\n)
COMMENT 'Parsed FHIR Patient Records'
TBLPROPERTIES (
  'delta.enableChangeDataFeed' = 'true'
  ,'delta.enableDeletionVectors' = 'true'
  ,'delta.enableRowTracking' = 'true'
  ,'quality' = 'bronze'
  ,'pipelines.channel' = 'PREVIEW'
  ,'delta.feature.variantType-preview' = 'supported'
)
AS SELECT *
FROM 
(
  SELECT
    resource_uuid as patient_uuid 
  , bundle_uuid
  , fullUrl as patient_url
  , key
  , value
  FROM 
    STREAM(fhir_bronze_resources)
  WHERE 
    resourceType = 'Patient'
)
PIVOT (first(value) FOR key IN ('"||patient_keys||"'));
"

In [0]:
EXECUTE IMMEDIATE patient_resource_stmnt;

In [0]:
select * from patient limit 10;